# 使用服務與工具
* Jupyter
* Python 3.6
* Azure CLI
* Azure VM
* Azure Cognitive Services
* Azure Custom Vision
* Azure Container Registry
* Azure IoT Hub
* Raspberry Pi

In [ ]:
# 1.1 安裝 custom vision 套件
!pip install azure-cognitiveservices-vision-customvision

In [ ]:
# 1.2 安裝 Azure CLI
!pip install azure-cli

In [ ]:
# 1.3 登入 Azure
!az login

In [ ]:
# 1.4 相關資訊設定
# 設定資源群組所在位置
resource_group_location="westus2"

# 設定資源群組名稱
date="$(date -d '8hour' +'%Y%m%d')"
resource_group="CustomVision"+date

In [ ]:
# 1.5 新建資源群組
!az group create -l $resource_group_location -n $resource_group

In [ ]:
# 1.6 建立 cognitive services - Training
trainingName = "CustomVision-Training"
!az cognitiveservices account create --name $trainingName --resource-group $resource_group --kind CustomVision.Training --sku S0 --location $resource_group_location --yes

In [ ]:
# 1.7 取得 Training key
training_key = !az cognitiveservices account keys list --name $trainingName --resource-group $resource_group | grep -o '"key1": "[^"]*' | grep -o '[^"]*$'
!echo $training_key 

In [ ]:
# 1.8 取得 cognitive services endpoint
endpoint = !az cognitiveservices account list --resource-group $resource_group | grep -o '"endpoint": "[^"]*' | grep -o '[^"]*$'
!echo $endpoint

In [ ]:
# 1.9 建立專案
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry

trainer = CustomVisionTrainingClient(training_key[0], endpoint= endpoint[0])
for domain in trainer.get_domains():
  print(domain.id, "\t", domain.name) 

project = trainer.create_project("Lego", domain_id = "0732100f-1a38-4e49-a514-c9b44c697ab5")

In [ ]:
# 1.10 上傳 banana 影像到專案
print ("Adding banana images...")
banana_tag = trainer.create_tag(project.id, "banana")
import os
from multiprocessing.dummy import Pool as ThreadPool
def upload(filepath):
    with open(filepath, mode="rb") as img_data:
        trainer.create_images_from_data(project.id, img_data.read(), [ banana_tag.id ])
    
def upload_parallel(filepaths, threads=5):
    pool = ThreadPool(threads)
    results = pool.map(upload, filepaths)
    pool.close()
    pool.join()
valid_dir = "./Fruit/banana/"
valid_paths = [valid_dir + os.fsdecode(image) for image in os.listdir(os.fsencode(valid_dir))]
img_paths = valid_paths
upload_parallel(img_paths)
print ("Adding banana images Finish ...")

In [ ]:
# 1.11 上傳 guava 影像到專案
print ("Adding guava images...")
guava_tag = trainer.create_tag(project.id, "guava")
def upload(filepath):
    with open(filepath, mode="rb") as img_data:
        trainer.create_images_from_data(project.id, img_data.read(), [ guava_tag.id ])
    
def upload_parallel(filepaths, threads=5):
    pool = ThreadPool(threads)
    results = pool.map(upload, filepaths)
    pool.close()
    pool.join()
valid_dir = "./Fruit/guava/"
valid_paths = [valid_dir + os.fsdecode(image) for image in os.listdir(os.fsencode(valid_dir))]
img_paths = valid_paths
upload_parallel(img_paths)
print ("Adding guava images Finish ...")

In [ ]:
# 1.12 開始訓練
import time
print ("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(1)

In [ ]:
# 1.13 建立 cognitive services - Prediction
predictionName = "CustomVision-Prediction"
prediction_resource_id = !az cognitiveservices account create --name $predictionName --resource-group $resource_group --kind CustomVision.Prediction --sku S0 --location $resource_group_location --yes | grep -o '"id": "[^"]*' | grep -o '[^"]*$' 

In [ ]:
# 1.14 Publish Model
publish_iteration_name = "classifyModel"
trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id[0])
print ("Publish Model Done!")  

In [ ]:
# 1.15 取得 Prediction key
prediction_key = !az cognitiveservices account keys list --name $predictionName --resource-group $resource_group | grep -o '"key1": "[^"]*' | grep -o '[^"]*$'
!echo $prediction_key 

In [ ]:
# 1.16 取得 prediction Endpoint
predictionEndpoint = !az cognitiveservices account show --name $predictionName --resource-group $resource_group | grep -o '"endpoint": "[^"]*' | grep -o '[^"]*$'
!echo $predictionEndpoint

In [ ]:
# 1.17 測試
import requests
import json

from os import walk
from os.path import join

predictor = CustomVisionPredictionClient(prediction_key, endpoint=predictionEndpoint)

canPath = './Fruit/test'

for root, dirs, files in walk(scanPath):
  for f in files:
    fullpath = join(root, f)
    fileName = os.path.basename(fullpath)
    print("fullpath: %s, fileName: %s" %(fullpath, fileName))
    url="https://westus2.api.cognitive.microsoft.com/customvision/v3.0/Prediction/" + project.id + "/classify/iterations/classifyModel/image"
    headers={
        'content-type':'application/octet-stream',
        'Prediction-Key': prediction_key[0]
    }
    r =requests.post(url,data=open(fullpath,"rb"),headers=headers)
    results = r.json()['predictions']
    for prediction in results:
        print( prediction['tagName'] + ": {0:.2f}%".format(prediction['probability'] * 100))
    print("\n")

In [ ]:
# 1.18 建立訓練完成的 Model 下載連結
import requests
import json

url = endpoint[0] + "customvision/v3.0/training/projects/" + project.id + "/iterations/" + iteration.id + "/export?platform=DockerFile&flavor=Linux"
payload = {}

headers = {
    "Training-Key": training_key[0]
}
r = requests.post(url, data=json.dumps(payload), headers=headers)
print(r.json())

In [ ]:
# 1.19 取得訓練完成的 Model 下載連結
url = endpoint[0] + "/customvision/v3.0/training/projects/" + project.id + "/iterations/" + iteration.id + "/export"
headers = {
    "Training-Key": training_key[0]
}
r = requests.get(url, headers=headers)
downloadUri = r.json()[0]['downloadUri']
print(downloadUri)

In [ ]:
# 1.20 安裝 wget 套件
!pip install wget

In [ ]:
# 1.21 下載訓練完成的 Model
import wget
modelZipName = "cognitive-services-linux-dockerfile"
wget.download(downloadUri, "./" + modelZipName)

In [ ]:
# 1.22 安裝 unzip 套件
!sudo apt-get install unzip

In [ ]:
# 1.23 解壓縮 Model 
unzipName = modelZipName + "-model"
!unzip $modelZipName -d $unzipName

In [ ]:
# 1.24 刪除 cognitive services
!az cognitiveservices account delete --name $trainingName --resource-group $resource_group 
!az cognitiveservices account delete --name $predictionName --resource-group $resource_group 